In [1]:
import os
os.chdir("/private/home/yuchenlin/SemanticDebugger")

import json 
import altair as alt 
import warnings
import pandas
warnings.filterwarnings('ignore')

upstream_ex_ids = set()

def show_memory_ratio(path, prefix, ): 
    print(path)
    if len(upstream_ex_ids) == 0:
        with open("exp_results/data_streams/mrqa.nq_train.memory.jsonl") as f:
            lines = f.read().splitlines()
            for line in lines:
                item = json.loads(line)
                upstream_ex_ids.add(item["id"]) 

    data = json.load(open(path)) 
    ratios = []
    for time_item in  data["online_eval_results"]:
        if "retrieved_ids" in time_item:
            retrieved_ids = set(time_item["retrieved_ids"])
            upstream_ratio = len(retrieved_ids.intersection(upstream_ex_ids)) / len(retrieved_ids)
            ratios.append({"prefix":prefix, "timecode": time_item["timecode"], "upstream_ratio": upstream_ratio, "online_ratio": 1-upstream_ratio})
    # print(ratios)
    return ratios



def draw_curve(df, y_scale=[0, 1], fig_title="", y_title="EM", y_key="em:Q", height=800):
    x = alt.X("timecode", type="ordinal", title="Timecode")
    # y_em = alt.Y("em", type="quantitative", title="EM", scale=alt.Scale(domain=[0.5, 1.0]))
    # em_line = alt.Chart(df).mark_line(interpolate='natural', point=True).encode(x=x, y=y_em, opacity=alt.value(0.8), color=alt.value('red'))
    # scale = alt.Scale(domain=list(set(df["prefix"])), range=['red', 'orange', 'purple', 'blue', 'green'])
    # color=alt.Color('prefix:N', scale=scale)

    color=alt.Color('prefix:N') 
    
    # fig = alt.Chart(reformatted_data).mark_area(opacity=0.6).encode(x="timecode:O", y=alt.Y("em:Q", stack=None, title="EM"), color=color)
    
    # fig = alt.Chart(df).mark_line(opacity=0.7, interpolate="natural", point=True).encode(x=x, y=alt.Y("em:Q", stack=None, title="EM", scale=alt.Scale(domain=[0.4, 1])), color=color).properties(title="Knowledge Retain in EM acc. (forgetting measure) ")
  
    fig = alt.Chart(df).mark_line(opacity=0.7,  point=True).encode(x=x, y=alt.Y(y_key, stack=None, title=y_title, scale=alt.Scale(domain=y_scale)), color=color).properties(title=fig_title)

    fig = alt.layer(fig).resolve_scale()
    fig = fig.properties(width=1100, height=height).configure_axis(
        labelFontSize=18,
        titleFontSize=16, 
    ).configure_legend(titleFontSize=0, labelFontSize=15, orient='bottom-right', strokeColor='gray',
        fillColor='#EEEEEE',
        padding=10,
        cornerRadius=5,).configure_title(
        fontSize=20,
        font='Courier',
        anchor='middle',
        orient="top", align="center",
        color='black'
    )
    return fig


ratios = []
for seed in ["42"]:
    ratios += show_memory_ratio(f"exp_results/dynamic_stream/memory_based/results/1014_MixedAllErrors_T=100_er_M=U+I_rs=32_rq=3_seed={seed}_result.json", f"ER-T100-F3 ({seed})",)
    ratios += show_memory_ratio(f"exp_results/dynamic_stream/memory_based/results/1014_MixedAllErrors_T=100_mir_M=U+I_rs=32_rq=3_candidate=256_mode=none_seed={seed}_result.json", f"MIR-C256-T100-F3 ({seed})",)
    ratios += show_memory_ratio(f"exp_results/dynamic_stream/index_based/results/1001v2_MixedAllErrors_T=100_index_M=U+I_rs=32_rq=3_rank=most_similar_mir=no(0)_seed={seed}_result.json", f"BiEncoder ({seed})",)


ratios = pandas.DataFrame(ratios)

draw_curve(df=ratios, fig_title="online_ratio", y_scale=[0., 1], y_key="online_ratio")

# .save('figures/cf_kr_curve.png')

exp_results/dynamic_stream/memory_based/results/1014_MixedAllErrors_T=100_er_M=U+I_rs=32_rq=3_seed=42_result.json
exp_results/dynamic_stream/memory_based/results/1014_MixedAllErrors_T=100_mir_M=U+I_rs=32_rq=3_candidate=256_mode=none_seed=42_result.json
exp_results/dynamic_stream/index_based/results/1001v2_MixedAllErrors_T=100_index_M=U+I_rs=32_rq=3_rank=most_similar_mir=no(0)_seed=42_result.json


alt.LayerChart(...)

In [2]:
! pwd

/private/home/yuchenlin/SemanticDebugger/semanticdebugger/notebooks
